In [4]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer", "viridis")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
#source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_grid_search.R")
source("~/oceanography/scripts/LL_biophys.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R")
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained
source("~/oceanography/scripts/format_genetic_parentage_matrix.R")
source("~/oceanography/scripts/format_genetic_kernel_parentage_matrix.R")
source("~/oceanography/scripts/format_biophys_normalized_matrix.R")
source("~/oceanography/scripts/format_biophys_parentage_matrix.R")

#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the necessary input data for likelihood functions
load(file= "~/oceanography/script_output/SurveyData/for_likelihood_functions/2021-11-04_InputSurveyData.Rdata")
load(file= "~/oceanography/script_output/ROMSDataTables/2021-11-04_SeededParticleInfo.Rdata")
load(file="~/oceanography/script_output/SurveyData/for_likelihood_functions/2021-11-04_AddDestTables.Rdata")

In [5]:
TotalPar2012_4 <- fread(file="~/oceanography/empirical_data/genetics/parentage_table_2012-14.csv")
TotalPar2012_4 <- unique(TotalPar2012_4[offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][ #collapse Magbangon values
            , n_matches := sum(n_matches), by=c("offs_site", "par_site", "year")], by=c("offs_site", "par_site", "year"))

MonsoonRecSampPar <- unique(fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")[
    , .(offs_site=site, par_site, monsoon=season)][offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][ #collapse Magbangon values
    , num_gen := (.N), by=c("offs_site", "monsoon")][
    !is.na(par_site), n_matches := as.numeric((.N)), by=c("offs_site", "par_site", "monsoon")], by=c("offs_site", "par_site", "monsoon"))[is.na(n_matches), n_matches:=0]

genetic_parentage_matrices <- format_genetic_parentage_matrix(TotalPar2012_4, MonsoonRecSampPar, SurveyData)
#fwrite(genetic_parentage_matrices$GenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2012, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2013, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2014, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixSWM2012-14ForROMSComp.csv")


genetic_kernel_matrices <- format_genetic_kernel_parentage_matrix(TotalPar2012_4, MonsoonRecSampPar, SurveyData)

#fwrite(genetic_kernel_matrices$KernelGenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2012, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2013, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2014, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv")


__read in the biophysical data and format for likelihood functions__

In [6]:
SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableCompleteMetaLongForm08DayPLD.csv")[year %in% c(2012, 2013, 2014) & dest != "CAI"] #filter out CAI as a destination for now, not very well spatially defined
total_release_days

[1] 687

In [16]:

biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConn, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestAnnualSim=AddDestAnnualSim, AddDestMonsoonSim=AddDestMonsoonSim)

#fwrite(biophys_norm_matrices$FullBiophysMatNorm, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2012, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2012ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2013, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2013ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2014, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2014ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormNEM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrixNEM2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormSWM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrixSWM2012-14ForROMSComp15DayPLD.csv")

biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConn, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestGenAnnual=AddDestGenAnnual)
#fwrite(biophys_parentage_matrices$FullBiophysMat, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2012, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2012ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2013, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2013ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2014, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2014ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatNEM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixNEM2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatSWM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixSWM2012-14ForROMSComp15DayPLD.csv")

In [129]:
#get the best-fit kernels and profile likelihood confidence intervals in fit_data.ipynb


In [ ]:

#bootstrap the particle AND parentage data and re-fit kernels and perform bode likelihood function for dispersal event matching
num_samples_sim <- nrow(SimConn) #because each row in SimConn is a particle the number of rows is N samples per iteration for bootstrapping
#num_samples_par <- sum(genetic_kernel_matrices$KernelGenMat2012_4[1:nrow(genetic_kernel_matrices$KernelGenMat2012_4)-1,]) #N samples per iteration for bootstrapping the genetic parentage data is the total number of recruits samp
num_samples_unassigned <- sum(genetic_kernel_matrices$KernelGenMat2012_4[nrow(genetic_kernel_matrices$KernelGenMat2012_4),]) #N samples per iteration for bootstrapping the genetic parentage data is the total number of recruits sampled
num_samples_monsoon_genotyped <- sum(unique(MonsoonRecSampPar, by=c("offs_site", "monsoon"))[, num_gen])
num_samples_monsoon_par <- sum(unique(MonsoonRecSampPar, by=c("offs_site","par_site", "monsoon"))[, n_matches])

#make table to hold results
BootKernels <- data.table(data_source=character(), time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), log_like=numeric(), iteration=numeric())
#BootKernels <- as.data.frame(matrix(NA, nrow = 0, ncol = 9))
#setDT(BootKernels)
#setnames(BootKernels, c("data_source", "time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))
BootLikelihood <- data.table(comparison=character(), time_scale=character(), log_like=numeric(), iteration=numeric())

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

for(i in 1:1000){
    
    #resample dispersal data, both genetics and simulation
    SimConnInt2 <- SimConn[, .SD[sample(.N, num_samples_sim, replace=TRUE)]] #sample the particle data
    AllFishObsWithPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")[year %in% c("2012", "2013", "2014")][offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"] #collapse Magbangon values
 
    #make parentage tables long form to sample rows for bootstrapping
    TotalParLongFormInt2 <- AllFishObsWithPar[matched_offs =="Y", num_matches_annual := .N, by=c("year", "offs_site", "par_site")][
        is.na(num_matches_annual), num_matches_annual := 0][
        year %in% c("2012", "2013", "2014"), .SD[sample(.N, num_matches_annual, replace=TRUE)], by=c("year", "offs_site", "par_site")][
        ,-"num_matches_annual"]#num_matches_annual is used as the number of rows to draw for each source/dest/year combo, then make a summary table and drop that column now that the data is sampled 
    #nrow(TotalParLongFormInt2[matched_offs=="Y"]) #should be 37
    TotalPar2012_4Int3 <- TotalParLongFormInt2[, .(n_matches=.N), by=c("offs_site", "par_site", "year")] 
    
    AllRecruitsSampledLongForm <- AllFishObsWithPar[is.na(par_site)][, num_unassigned_annual := .N, by=c("year", "site")][
        is.na(num_unassigned_annual), num_unassigned_annual := 0][
        , .SD[sample(.N, num_unassigned_annual, replace=TRUE)], by=c("year", "site")][
        ,-c("num_unassigned_annual", "num_matches_annual")]#num_sampled_annual is used as the number of rows to draw for each dest/year combo, then make a summary table and drop that column now that the data is sampled 
    #nrow(AllRecruitsSampledLongForm)#should be 357
    
    AllRecruitsSampledInt <- rbind(TotalParLongFormInt2, AllRecruitsSampledLongForm)[,.(n_offs_gen=.N), by=c("year", "site")] #use the values in this table to re-make SurveyData
    #sum(AllRecruitsSampledInt$n_offs_gen)#should be 394
    SurveyDataInt <- left_join(SurveyData[year %in% c("2012", "2013", "2014"),-"n_offs_gen"], AllRecruitsSampledInt, by=c("year", "site"))[is.na(n_offs_gen), n_offs_gen := 0] 
    #sum(SurveyDataInt$n_offs_gen) #should be 394
    
    MonsoonRecSampParLongForm <- MonsoonRecSampPar[rep(seq_len(nrow(MonsoonRecSampPar)), MonsoonRecSampPar$n_matches), 1:ncol(MonsoonRecSampPar)][, -"n_matches"]
    #nrow(SurveyDataLongForm) #should be 357
    #nrow(TotalParLongForm)#should be 37
    #nrow(MonsoonRecSampParLongForm) #should be 17   
    

    
    #for monsoon genotyped recruits
    MonsoonRecSampLongForm <- MonsoonRecSampPar[rep(seq_len(nrow(MonsoonRecSampPar)), MonsoonRecSampPar$num_gen), 1:ncol(MonsoonRecSampPar)][, -c("num_gen", "n_matches")]
    MonsoonRecSampLongFormInt2 <- MonsoonRecSampLongForm[, .SD[sample(.N, num_samples_monsoon_genotyped, replace=TRUE)]]
    
    #for monsoon assigned
    MonsoonRecSampParLongForm <- MonsoonRecSampPar[rep(seq_len(nrow(MonsoonRecSampPar)), MonsoonRecSampPar$n_matches), 1:ncol(MonsoonRecSampPar)][, -c("num_gen", "n_matches")]   
    MonsoonRecSampParLongFormInt2 <- MonsoonRecSampParLongForm[, .SD[sample(.N, num_samples_monsoon_par, replace=TRUE)]]
    
    #check
    #nrow(MonsoonRecSampParLongFormInt)+nrow(MonsoonRecSampLongFormInt)#should be 273 because the parentage assignments are included in the num_genotyped for this input table
    
    #turn back into summary table
    MonsoonRecSampParInt3 <- left_join(MonsoonRecSampLongFormInt2[, .(num_gen=.N), by=c("offs_site",  "monsoon")], MonsoonRecSampParLongFormInt2[, .(n_matches=.N), by=c("offs_site", "par_site", "monsoon")], by=c("offs_site", "monsoon"))[, n_matches := as.numeric(n_matches)]    

        
        #format the matrices for kernel fitting and likelihood function
        genetic_parentage_matrices <- format_genetic_parentage_matrix(TotalPar2012_4=TotalPar2012_4Int3, MonsoonRecSampPar=MonsoonRecSampParInt3, SurveyData=SurveyDataInt)
        genetic_kernel_matrices <- format_genetic_kernel_parentage_matrix(TotalPar2012_4=TotalPar2012_4Int3, MonsoonRecSampPar=MonsoonRecSampParInt3, SurveyData=SurveyDataInt)
        biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestAnnualSim=AddDestAnnualSim, AddDestMonsoonSim=AddDestMonsoonSim)
        biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestGenAnnual=AddDestGenAnnual)



#fit the kernels
    #fit the kernels, get the biophysical data together
biophys_par_data2012 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$FullBiophysMat), ncol=1, 1))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))

#genetic data with same reef metadata structure as model

par_data2012 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))


#make a summary table for all time frames of simulation data
SimKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 9))
setDT(SimKernelsInt)
setnames(SimKernelsInt, c("data_source", "time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[1] <- i
SimKernelsInt$data_source[1] <- "simulation"
SimKernelsInt$time_scale[1] <- "annual"
SimKernelsInt$time_id[1] <- "2012"
SimKernelsInt$k[1] <-  BestK2012
SimKernelsInt$theta[1] <- BestTheta2012
SimKernelsInt$mdd[1] <- MDD2012
SimKernelsInt$med[1] <- Med2012
SimKernelsInt$dist90[1] <- Dist90_2012
SimKernelsInt$log_like[1] <- logLik(Sim2012Fit)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[2] <- i
SimKernelsInt$data_source[2] <- "simulation"
SimKernelsInt$time_scale[2] <- "annual"
SimKernelsInt$time_id[2] <- "2013"
SimKernelsInt$k[2] <-  BestK2013
SimKernelsInt$theta[2] <- BestTheta2013
SimKernelsInt$mdd[2] <- MDD2013
SimKernelsInt$med[2] <- Med2013
SimKernelsInt$dist90[2] <- Dist90_2013
SimKernelsInt$log_like[2] <- logLik(Sim2013Fit)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[3] <- i
SimKernelsInt$data_source[3] <- "simulation"
SimKernelsInt$time_scale[3] <- "annual"
SimKernelsInt$time_id[3] <- "2014"
SimKernelsInt$k[3] <-  BestK2014
SimKernelsInt$theta[3] <- BestTheta2014
SimKernelsInt$mdd[3] <- MDD2014
SimKernelsInt$med[3] <- Med2014
SimKernelsInt$dist90[3] <- Dist90_2014
SimKernelsInt$log_like[3] <- logLik(Sim2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)
    
SimKernelsInt$iteration[4] <- i
SimKernelsInt$data_source[4] <- "simulation"
SimKernelsInt$time_scale[4] <- "total_years"
SimKernelsInt$time_id[4] <- "2012-4"
SimKernelsInt$k[4] <-  BestK2012_4
SimKernelsInt$theta[4] <- BestTheta2012_4
SimKernelsInt$mdd[4] <- MDD2012_4
SimKernelsInt$med[4] <- Med2012_4
SimKernelsInt$dist90[4] <- Dist90_2012_4
SimKernelsInt$log_like[4] <- logLik(Sim2012_4Fit)[1]

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[5] <- i
SimKernelsInt$data_source[5] <- "simulation"
SimKernelsInt$time_scale[5] <- "monsoonal"
SimKernelsInt$time_id[5] <- "NEM"
SimKernelsInt$k[5] <-  BestKNEM
SimKernelsInt$theta[5] <- BestThetaNEM
SimKernelsInt$mdd[5] <- MDDNEM
SimKernelsInt$med[5] <- MedNEM
SimKernelsInt$dist90[5] <- Dist90_NEM
SimKernelsInt$log_like[5] <- logLik(SimNEMFit)[1]

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[6] <- i
SimKernelsInt$data_source[6] <- "simulation"
SimKernelsInt$time_scale[6] <- "monsoonal"
SimKernelsInt$time_id[6] <- "SWM"
SimKernelsInt$k[6] <-  BestKSWM
SimKernelsInt$theta[6] <- BestThetaSWM
SimKernelsInt$mdd[6] <- MDDSWM
SimKernelsInt$med[6] <- MedSWM
SimKernelsInt$dist90[6] <- Dist90_SWM
SimKernelsInt$log_like[6] <- logLik(SimSWMFit)[1]



#make a summary table for all time frames of genetic data
GenKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 9))
setDT(GenKernelsInt)
setnames(GenKernelsInt, c("data_source", "time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))
#GenKernelsInt <- data.table(data_source=time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Gen2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Gen2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[1] <- i
GenKernelsInt$data_source[1] <- "genetics"    
GenKernelsInt$time_scale[1] <- "annual"
GenKernelsInt$time_id[1] <- "2012"
GenKernelsInt$k[1] <-  BestK2012
GenKernelsInt$theta[1] <- BestTheta2012
GenKernelsInt$mdd[1] <- MDD2012
GenKernelsInt$med[1] <- Med2012
GenKernelsInt$dist90[1] <- Dist90_2012
GenKernelsInt$log_like[1] <- logLik(Gen2012Fit)[1]

BestK2013 <- as.numeric(coef(Gen2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Gen2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[2] <- i
GenKernelsInt$data_source[2] <- "genetics"    
GenKernelsInt$time_scale[2] <- "annual"
GenKernelsInt$time_id[2] <- "2013"
GenKernelsInt$k[2] <-  BestK2013
GenKernelsInt$theta[2] <- BestTheta2013
GenKernelsInt$mdd[2] <- MDD2013
GenKernelsInt$med[2] <- Med2013
GenKernelsInt$dist90[2] <- Dist90_2013
GenKernelsInt$log_like[2] <- logLik(Gen2013Fit)[1]

BestK2014 <- as.numeric(coef(Gen2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Gen2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[3] <- i
GenKernelsInt$data_source[3] <- "genetics"    
GenKernelsInt$time_scale[3] <- "annual"
GenKernelsInt$time_id[3] <- "2014"
GenKernelsInt$k[3] <-  BestK2014
GenKernelsInt$theta[3] <- BestTheta2014
GenKernelsInt$mdd[3] <- MDD2014
GenKernelsInt$med[3] <- Med2014
GenKernelsInt$dist90[3] <- Dist90_2014
GenKernelsInt$log_like[3] <- logLik(Gen2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Gen2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Gen2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[4] <- i
GenKernelsInt$data_source[4] <- "genetics"    
GenKernelsInt$time_scale[4] <- "total_years"
GenKernelsInt$time_id[4] <- "2012-4"
GenKernelsInt$k[4] <-  BestK2012_4
GenKernelsInt$theta[4] <- BestTheta2012_4
GenKernelsInt$mdd[4] <- MDD2012_4
GenKernelsInt$med[4] <- Med2012_4
GenKernelsInt$dist90[4] <- Dist90_2012_4
GenKernelsInt$log_like[4] <- logLik(Gen2012_4Fit)[1]

BestKNEM <- as.numeric(coef(GenNEMFit)[1])
BestThetaNEM <- as.numeric(coef(GenNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[5] <- i
GenKernelsInt$data_source[5] <- "genetics"    
GenKernelsInt$time_scale[5] <- "monsoonal"
GenKernelsInt$time_id[5] <- "NEM"
GenKernelsInt$k[5] <-  BestKNEM
GenKernelsInt$theta[5] <- BestThetaNEM
GenKernelsInt$mdd[5] <- MDDNEM
GenKernelsInt$med[5] <- MedNEM
GenKernelsInt$dist90[5] <- Dist90_NEM
GenKernelsInt$log_like[5] <- logLik(GenNEMFit)[1]

BestKSWM <- as.numeric(coef(GenSWMFit)[1])
BestThetaSWM <- as.numeric(coef(GenSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[6] <- i
GenKernelsInt$data_source[6] <- "genetics"    
GenKernelsInt$time_scale[6] <- "monsoonal"
GenKernelsInt$time_id[6] <- "SWM"
GenKernelsInt$k[6] <-  BestKSWM
GenKernelsInt$theta[6] <- BestThetaSWM
GenKernelsInt$mdd[6] <- MDDSWM
GenKernelsInt$med[6] <- MedSWM
GenKernelsInt$dist90[6] <- Dist90_SWM
GenKernelsInt$log_like[6] <- logLik(GenSWMFit)[1]

GenKernelsInt$iteration <- i
GenKernelsInt$data_source <- "genetics"

        BootKernelsInt <- rbind(SimKernelsInt, GenKernelsInt)
    
        BootKernels <- rbind(BootKernels, BootKernelsInt)
    
 #also bootstrap the biophysical likelihood
    Data_par2012_bio2012 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012[1:nrow(genetic_parentage_matrices$GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012[nrow(genetic_parentage_matrices$GenMat2012),]))

Data_par2013_bio2013 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2013[1:nrow(genetic_parentage_matrices$GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2013[nrow(genetic_parentage_matrices$GenMat2013),]))

Data_par2014_bio2014 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2014[1:nrow(genetic_parentage_matrices$GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2014[nrow(genetic_parentage_matrices$GenMat2014),]))


#total compared to total
Data_par2012_4_bio2012_2_4 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012_4[1:nrow(genetic_parentage_matrices$GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012_4[nrow(genetic_parentage_matrices$GenMat2012_4),]))

#for seasonal kernels

 Data_parNEM_bioNEM <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),]))

 Data_parSWM_bioSWM <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))
#total to total 
 Data_parNEMSWM_bio2012_4 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=(as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(sampled_reefs_vec),])+
                                  as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(sampled_reefs_vec),])), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),])+
                    as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))

#calculate likelihood
LL_annual_annual <- LL_biophys(Data_par2012_bio2012)+LL_biophys(Data_par2013_bio2013)+LL_biophys(Data_par2014_bio2014)
LL_total_total <- LL_biophys(Data_par2012_4_bio2012_2_4)
LL_monsoonal_monsoonal <- LL_biophys(Data_parNEM_bioNEM)+LL_biophys( Data_parNEM_bioNEM)
LL_mon_total_total <- LL_biophys(Data_parNEMSWM_bio2012_4)

#store results
LLInt <- data.table(comparison= c("gen_total_bio_total", "gen_annual_bio_annual", "gen_mon_total_bio_mon_total", "gen_monsoonal_bio_mon_total"), 
           time_scale= c("total_years", "annual", "total_monsoons", "monsoonal"),
           log_like=c(LL_total_total, LL_annual_annual, LL_mon_total_total, LL_monsoonal_monsoonal))[order(-log_like)]

LLInt$iteration <- i
    
BootLikelihood <- rbind(BootLikelihood, LLInt)
    
    setTxtProgressBar(pb, i)
    }
close(pb)

fwrite(BootKernels, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalKernels08DayPLD.csv")
fwrite(BootLikelihood, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalLikelihoods08DayPLD.csv")

  |==                                                                    |   3%